In [10]:
import hashlib

Esta implementação seguiu a documentação pelo que, porque achamos que não vale a pena repetir o que está dito no documento da implementação, vamos falar apenas das duvidas e problemas que encontrá-mos ao longo do nosso trabalho.

Em primeiro lugar tivemos dúvidas na geração do parâmetro **cs** pois na documentação ele não está em notação de vetor mas parece-nos ter o comportamento de um. O mesmo se pode dizer para a variável **ct**.

Encontramos também algumas dificuldades nas funções auxiliares **aff** e **rainbowmap**. Mas as que provavelmente influenciaram mais na falha de verificação foi as funções **H** e **invF**.

Em primeiro lugar nós achamos que a função **H** não está implementada corretamente, o que nós fizemos foi usar a função hash do documento e de seguida transformamos os bytes resultantes num array de inteiros (0 ou 1) correspondente aos bits.

Quanto à função **invF**, não entendemos como é que podemos aplicar **y_v1** a **F** visto que têm tamanhos diferentes.

In [11]:
class Rainbow():    
  def __init__(self):
    # Parameter Set I
    self.q  = 16
    self.F  = GF(self.q) # Finite field with q elements
    self.v1 = 36
    self.o1 = 32
    self.o2 = 32
    self.n  = 100        # Number of variables
    self.m  = 64         # Number of equations


  # Helper Functions
  def hash_function(self, msg):
    def _access_bit(data, num):
      base  = int(num // 8)
      shift = int(num % 8)
      return (data[base] & (1 << shift)) >> shift

    shake = hashlib.shake_256()
    shake.update(msg)
    hashed = shake.digest(int(self.m / 8))
    bits   = [_access_bit(hashed, i) for i in range(self.m)]
    return matrix(bits)


  def aff(self, M, c):
    x = random_matrix(self.F, M.nrows(), 1)
    return M * x + c

  def rainbow_map(self):
    return random_matrix(self.F, 1, self.n)

  def inv_F(self, F, x):
    y_v1 = random_matrix(self.F, self.v1, 1)
    results = []
    for i in range(self.v1):
      f = (F[0][i]).polynomial()
      results.append(f(y_v1[i][0]))
    x = None
    return result


  # Key Generation
  def key_generation(self):
    # We create an invertible matrix Ms
    Ms = random_matrix(self.F, self.m, self.m)
    while Ms.is_invertible() == False:
      Ms = random_matrix(self.F, self.m, self.m)

    cs   = random_matrix(self.F, self.m, 1)
    S    = self.aff(Ms, cs)
    InvS = Ms.inverse()

    # We create an invertible matrix Mt
    Mt = random_matrix(self.F, self.n, self.n)
    while Ms.is_invertible() == False:
      Mt = random_matrix(self.F, self.n, self.n)
    
    ct   = random_matrix(self.F, self.n, 1)
    T    = self.aff(Mt, ct)
    InvT = Mt.inverse()

    F = self.rainbow_map()
    P = S * F * T

    secret_key = (InvS, cs, F, InvT, ct)
    public_key = P
    return secret_key, public_key


  # Sign
  def sign(self, secret_key, d):
    InvS, cs, F, InvT, ct = secret_key
    
    h = self.hash_function(d.encode())
    x = InvS * (h.transpose() - cs)
    #y = self.inv_F(F, x)
    y = random_matrix(self.F, 1, self.n)
    z = InvT * (y.transpose() - ct)

    return z

  # Verify
  def verify(self, d, z, public_key):
    P  = public_key
    h  = self.hash_function(d.encode())
    hl = P * z.transpose()
    if h == hl:
      print('Yo')
    else:
      print('Not Yo')


In [12]:
d = 'Lets pretende this is a document'
rainbow = Rainbow()
secret_key, public_key = rainbow.key_generation()
z = rainbow.sign(secret_key, d)
rainbow.verify(d, z, public_key)

Not Yo
